<a href="https://colab.research.google.com/github/10udCryp7/Speech-Practice/blob/main/notebooks/05_Eval_ASR_Whisper_Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install datasets

In [ ]:
!conda install evaluate

In [ ]:
!conda install jiwer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Load Model Pipeline

In [1]:
from transformers import pipeline

model_id = 'openai/whisper-small'
task = 'automatic-speech-recognition'

pipe = pipeline(task = task, model = model_id)

/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


# Load Dataset

In [2]:
from datasets import load_dataset

common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv", split="test"
)

In [3]:
sub_test = common_voice_test.shuffle(seed = 42).select(range(100))

# Infer

In [5]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset

prediction_list = []
for prediction in tqdm(
    pipe(
        KeyDataset(sub_test, 'audio'),
        generate_kwargs={"task": "transcribe", "max_new_tokens": 128},
        batch_size=16,
    ),
    total = len(sub_test)
):
  prediction_list.append(prediction['text'])

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is

In [6]:
labels = [sample['sentence'] for sample in sub_test]

# Evaluate

In [7]:
from evaluate import load


compute_metric = load('wer')

In [8]:
wer_score = compute_metric.compute(predictions = prediction_list, references = labels)

wer_score*100

340.4984423676012

# Eval Norm

In [13]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

labels_norm = [normalizer(label) for label in labels]
prediction_list_norm = [normalizer(prediction) for prediction in prediction_list]

wer_score_norm = compute_metric.compute(predictions = prediction_list_norm, references = labels_norm)

wer_score_norm*100

161.59278148293447